# 记住我们都会错，最重要的是要知道什么是对错，不要怕犯错，及时改正错误，不要被情绪干扰。


# 在买卖点应当观察主要货币的成交量和涨幅匹配程度
# 记得涨的时候容易涨的激进，跌的时候容易跌的激进。但是涨的时候不太容易跌的很激进，跌的时候也不容易涨的很激进。
# 买卖点的时候应做计划，应当分批，应当每个案对待。¶
# 不要杠杆，容易有最差的情况会出现的心态
# 应当有设置止损线和止盈线
# 细节处理很重要，要实现分批，多维度。要实现移动止损。要看信号。
# 要看均线，要看实时状态，要记住错了怎么办
# 降低了风险就是在降低机会，要学会适应风险，学会认错

# 采用三种方法，来确定量化交易
# 1，利用均线系统来确定动量与风险

# 2， 利用追踪止损来确定止损位置

# 3， 利用Prophet 来确定周期和幅度

In [1]:
import sys 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import os
sys.path.append("D:\工作\QuantRobot")
from UserInterfer.IntegrateTools.feature_engineer import CommonFeature
from config import Config

In [ ]:
import matplotlib.pyplot as plt
path = os.path.join(Config.base_dir, 'spot')
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
reverse_df = CommonFeature(path, Greed=False, Rm=True, reverse=True)
norm_df = CommonFeature(path, Greed=False, Rm=True, reverse=False)
fig, ax1 = plt.subplots( figsize=(20,12))
plt.title('市场指数变化情况')
ax2 = ax1.twinx()
ax1.plot(reverse_df.index, reverse_df['market_index'],label='reverse')
ax1.set_ylabel('reverse')
ax2.plot(norm_df.index, norm_df['market_index'],color='red',label='norm')
ax2.set_ylabel('norm')
plt.legend()
plt.show()

# 1， 首先看指数与均线的关系，均线是重要的参照对线，虽然有时候会被穿透。但是均线往往是可靠的支撑或压力。

In [ ]:
import talib
import numpy as np
# 需要检查的指数
test_df = norm_df
# 观察的周期
period = 200
# 示例数据
close_prices = test_df['market_index']

# 计算5日移动平均线
average7= talib.SMA(close_prices, timeperiod=7)
average21 = talib.SMA(close_prices, timeperiod=21)
average63 = talib.SMA(close_prices, timeperiod=63)
adjust_norm = (test_df['market_index']-average63)/average63
adjust_7 = (average7-average63)/average63
adjust_21 = (average21-average63)/average63

plt.figure(figsize=(15, 10))
plt.plot(adjust_norm[-period:],label='index')
plt.plot(adjust_7[-period:],label='7days')
plt.plot(adjust_21[-period:],label='21days')
plt.plot(test_df.index[-period:], [0]*period)
plt.legend()
plt.show()


# 2，观察指数追踪止损情况

In [ ]:
from FeatureEngineer.trailing_factor import trailing_atr_factor
# 时间参数
period = 100
#波动倍率参数，不得小于1
down, above = 1.2,1.3

trailing_df =trailing_atr_factor(reverse_df, down, above, 'market_index')
trailing_df[-period:].plot(y=['market_index','trailing'],figsize=(12,10))

In [ ]:
# 观察具体数字
norm_df.tail()

# 3，观察prophet对指数的预测情况

In [ ]:
# test_index name
test_index = reverse_df

from prophet import Prophet
test_index = test_index.reset_index()
test_index = test_index.rename(columns={"date":'ds','market_index':"y"})
test_index['ds'] = test_index['ds'].dt.tz_localize(None)
# 指数预测
model = Prophet(changepoint_prior_scale=0.05,
                seasonality_mode='multiplicative',
               )
model.fit(test_index)
future = model.make_future_dataframe(periods=150)
forecast = model.predict(future)
model.plot(forecast)

# 二、 观察具体tic的情况

In [ ]:
# 载入所有的数据
from DataManager.data_prepare import clean_datas
path = os.path.join(Config.base_dir, 'spot')
current_time = pd.Timestamp.now()
# path = Config.base_dir
df = pd.DataFrame()
for file in os.listdir(path):
#     print(os.path.join(path,file))
    temp = pd.read_csv(os.path.join(path,file),index_col=0)
    temp = clean_datas(temp)
    temp['y'] = temp['close']  
    temp['date'] = pd.to_datetime(temp['date'])
    temp['date'] = temp['date'].dt.tz_localize(None)
    if temp['date'].iloc[-1] + pd.Timedelta(days=1)< current_time:       
        continue
    temp['tic'] = file[:-4]
    df = df.append(temp)
df = df.rename({'date':"ds"},axis=1)



In [ ]:
# 需要观察的tic
tic = 'ALDOGEUSDT'
object_df = df[df['tic']==tic]
# 观察的周期
period = 800
object_df = object_df.dropna()
object_df

# 1，单个股票的均线系统

In [ ]:

close_prices = object_df['y']
# 计算5日移动平均线
average7= talib.SMA(close_prices, timeperiod=7)
average21 = talib.SMA(close_prices, timeperiod=21)
average63 = talib.SMA(close_prices, timeperiod=63)
adjust_norm = (object_df['y']-average63)/average63
adjust_7 = (average7-average63)/average63
adjust_21 = (average21-average63)/average63

plt.figure(figsize=(15, 10))
plt.plot(object_df['ds'][-period:],adjust_norm[-period:],label=tic)
plt.plot(object_df['ds'][-period:],adjust_7[-period:],label='7days')
plt.plot(object_df['ds'][-period:],adjust_21[-period:],label='21days')
plt.plot( object_df['ds'][-period:],[0]*period)
plt.legend()
plt.show()

# 2，单个的追踪止损系统

In [ ]:
period = 100
down, above = 1.2,1.3

trailing_df =trailing_atr_factor(object_df, down, above, 'y')
trailing_df[-period:].plot(y=['y','trailing'],figsize=(12,10))

# 3，prophet 预测

In [ ]:
from prophet import Prophet
model = Prophet(changepoint_prior_scale=0.1,
                seasonality_mode='multiplicative',
                seasonality_prior_scale=10,
                changepoint_range = 0.9)
model = model.fit(object_df)
future = model.make_future_dataframe(periods=120)
forecast = model.predict(future)
model.plot(forecast)
# model.plot(forecast.iloc[-120:,])

In [ ]:
# from prophet.diagnostics import cross_validation
# df_cv = cross_validation(model, initial='500 days', period='60 days', horizon = '60 days')
# from prophet.diagnostics import performance_metrics
# df_p = performance_metrics(df_cv, rolling_window=1)
# df_p

In [ ]:
from prophet import Prophet
model = Prophet()
model = model.fit(object_df)
future = model.make_future_dataframe(periods=120)
forecast = model.predict(future)
model.plot(forecast)